In [3]:
import pandas as pd

In [5]:
root_path = './dataset/'

In [7]:
comportamiento_df = pd.read_csv(f'{root_path}comportamiento_tarjetasvisa.csv', sep = ',')

In [9]:
informacion_df = pd.read_csv(f'{root_path}informacion_adicional_tarjetas.csv', sep = ',')

In [11]:
comportamiento_df['ID_CLIENTE'].value_counts().shape[0]

45985

In [13]:
comportamiento_df.drop_duplicates()
comportamiento_df

,ID_CLIENTE,MESES,DIAS_VENCIDOS
0,5001711,0,NaN
1,5001711,-1,11.0
2,5001711,-2,16.0
3,5001711,-3,16.0
4,5001712,0,0.0
...,...,...,...
1048570,5150487,-25,0.0
1048571,5150487,-26,0.0
1048572,5150487,-27,0.0
1048573,5150487,-28,0.0


In [15]:
comportamiento_df.isnull().sum()

ID_CLIENTE            0
MESES                 0
DIAS_VENCIDOS    209230
dtype: int64

In [ ]:
#20% de las observaciones son nulas en columna DIAS_VENCIDOS

In [41]:
grupo_df = comportamiento_df.groupby(["ID_CLIENTE","MESES"]).max()
grupo

DIAS_VENCIDOS
ID_CLIENTE MESES               
5001711    -3              16.0
           -2              16.0
           -1              11.0
            0               NaN
5001712    -18             17.0
...                         ...
5150487    -4               0.0
           -3               0.0
           -2               0.0
           -1               0.0
            0               0.0

[1048575 rows x 1 columns]

In [17]:
filtro_nulos = comportamiento_df.groupby('ID_CLIENTE').filter(lambda x:x['DIAS_VENCIDOS'].isnull().any())
filtro_nulos['ID_CLIENTE'].value_counts()

ID_CLIENTE
5059942    61
5005201    61
5002806    61
5105565    61
5017058    61
           ..
5047681     1
5021959     1
5125464     1
5092092     1
5137027     1
Name: count, Length: 25005, dtype: int64

In [19]:
comportamiento_df['ID_CLIENTE'].value_counts()

ID_CLIENTE
5016769    61
5002806    61
5118192    61
5145767    61
5078567    61
           ..
5028795     1
5148442     1
5053900     1
5046332     1
5079047     1
Name: count, Length: 45985, dtype: int64

In [ ]:
# De un total de 45,985 clientes con comportamiento del manejo de tarjeta, 25,005 clientes tienen al menos un campo nulo en dias vencidos.

In [ ]:
## Se consideran todas las observaciones para definir el tipo de pagador toda vez que podemos asegurar un mal comportamiento con información cierta.

In [ ]:
## Score de comportamiento:
Mayor que 25 y menor igual que 30, se puntúa el comportamiento con 0.25.
Mayor que 30, se puntúa con 1.

In [21]:
# funcion para score comportamiento
def score_comportamiento(dias_vencidos):
    if dias_vencidos > 25 and dias_vencidos <= 30:
        return 0.25
    elif dias_vencidos > 30:
        return 1
    else:
        return 0

In [23]:
comportamiento_df['SCORE'] = comportamiento_df['DIAS_VENCIDOS'].apply(score_comportamiento)
comportamiento_df['MESES_ANTIGUEDAD'] = (comportamiento_df.groupby('ID_CLIENTE')['MESES'].transform('min')) * (-1)
comportamiento_df

,ID_CLIENTE,MESES,DIAS_VENCIDOS,SCORE,MESES_ANTIGUEDAD
0,5001711,0,NaN,0.0,3
1,5001711,-1,11.0,0.0,3
2,5001711,-2,16.0,0.0,3
3,5001711,-3,16.0,0.0,3
4,5001712,0,0.0,0.0,18
...,...,...,...,...,...
1048570,5150487,-25,0.0,0.0,29
1048571,5150487,-26,0.0,0.0,29
1048572,5150487,-27,0.0,0.0,29
1048573,5150487,-28,0.0,0.0,29


In [25]:
calificacion_df = comportamiento_df[["ID_CLIENTE","MESES_ANTIGUEDAD","SCORE"]].groupby(["ID_CLIENTE","MESES_ANTIGUEDAD"]).sum().reset_index()
calificacion_df

,ID_CLIENTE,MESES_ANTIGUEDAD,SCORE
0,5001711,3,0.00
1,5001712,18,0.25
2,5001713,21,0.00
3,5001714,14,0.00
4,5001715,59,0.00
...,...,...,...
45980,5150482,28,0.25
45981,5150483,17,0.00
45982,5150484,12,0.50
45983,5150485,1,0.25


In [27]:
# definir etiqueta de mal pagador
def tipo_cliente(meses_antiguedad, score_total):
    if meses_antiguedad <= 6 and score_total > 0:
        return 'mal pagador'
    elif meses_antiguedad > 6 and meses_antiguedad <= 12 and score_total > 0.5:
        return 'mal pagador'
    elif meses_antiguedad > 12 and meses_antiguedad <= 24 and score_total >= 1.5:
        return 'mal pagador'
    elif meses_antiguedad > 24 and meses_antiguedad <= 36 and score_total >= 2.5:
        return 'mal pagador'
    elif meses_antiguedad > 36 and meses_antiguedad <= 48 and score_total >= 3.5:
        return 'mal pagador'
    elif meses_antiguedad > 48 and meses_antiguedad <= 60 and score_total >= 4.5:
        return 'mal pagador'
    elif meses_antiguedad > 60 and score_total >= 5.5:
        return 'mal pagador'
    else:
        return 'buen pagador'

In [29]:
calificacion_df['TIPO_CLIENTE'] = calificacion_df.apply(lambda x: tipo_cliente(x['MESES_ANTIGUEDAD'], x['SCORE']), axis=1)
calificacion_df['TIPO_CLIENTE'].value_counts()

TIPO_CLIENTE
buen pagador    41728
mal pagador      4257
Name: count, dtype: int64